In [ ]:
# Baixar a versão mais recente do Google Cloud SDK
!curl -O https://dl.google.com/dl/cloudsdk/channels/rapid/downloads/google-cloud-sdk-430.0.0-linux-x86_64.tar.gz

# Extrair o arquivo baixado
!tar -xf google-cloud-sdk-430.0.0-linux-x86_64.tar.gz

# Instalar o SDK sem prompts interativos
!./google-cloud-sdk/install.sh -q

# Adicionar o gcloud ao PATH para facilitar o uso
import os
os.environ["PATH"] += ":/content/google-cloud-sdk/bin"

In [ ]:
from google.colab import auth
auth.authenticate_user()

# Configurar as credenciais de aplicação padrão para o gcloud
!gcloud auth application-default login --quiet

In [ ]:
# Substitua pelos seus valores
PROJECT_ID = '...'
ZONE = '...'
CLUSTER_NAME = '...'

# Obter credenciais
!./google-cloud-sdk/bin/gcloud container clusters get-credentials {CLUSTER_NAME} --zone {ZONE} --project {PROJECT_ID}

In [ ]:
# Baixar a versão mais recente do kubectl
!curl -LO "https://dl.k8s.io/release/$(curl -L -s https://dl.k8s.io/release/stable.txt)/bin/linux/amd64/kubectl"

# Tornar o kubectl executável
!chmod +x kubectl

# Mover o kubectl para um diretório no PATH
!sudo mv kubectl /usr/local/bin/

In [ ]:
# Verificar os nós do cluster
!kubectl get nodes

In [ ]:
deployment_yaml = """
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-deployment
spec:
  replicas: 2
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: nginx:stable
        ports:
        - containerPort: 80
        resources:
          requests:
            cpu: "100m"
          limits:
            cpu: "500m"
"""

with open('nginx-deployment.yaml', 'w') as f:
    f.write(deployment_yaml)

!kubectl apply -f nginx-deployment.yaml

In [ ]:
service_yaml = """
apiVersion: v1
kind: Service
metadata:
  name: nginx-service
spec:
  type: LoadBalancer
  selector:
    app: nginx
  ports:
    - protocol: TCP
      port: 80
      targetPort: 80
"""

with open('nginx-service.yaml', 'w') as f:
    f.write(service_yaml)

!kubectl apply -f nginx-service.yaml

In [ ]:
!kubectl apply -f https://github.com/kubernetes-sigs/metrics-server/releases/latest/download/components.yaml

In [ ]:
hpa_yaml = """
apiVersion: autoscaling/v2
kind: HorizontalPodAutoscaler
metadata:
  name: nginx-hpa
spec:
  scaleTargetRef:
    apiVersion: apps/v1
    kind: Deployment
    name: nginx-deployment
  minReplicas: 2
  maxReplicas: 5
  metrics:
  - type: Resource
    resource:
      name: cpu
      target:
        type: Utilization
        averageUtilization: 50
"""

with open('nginx-hpa.yaml', 'w') as f:
    f.write(hpa_yaml)

!kubectl apply -f nginx-hpa.yaml
hpa_yaml = """
apiVersion: autoscaling/v2
kind: HorizontalPodAutoscaler
metadata:
  name: nginx-hpa
spec:
  scaleTargetRef:
    apiVersion: apps/v1
    kind: Deployment
    name: nginx-deployment
  minReplicas: 2
  maxReplicas: 5
  metrics:
  - type: Resource
    resource:
      name: cpu
      target:
        type: Utilization
        averageUtilization: 50
"""

with open('nginx-hpa.yaml', 'w') as f:
    f.write(hpa_yaml)

!kubectl apply -f nginx-hpa.yaml

In [ ]:
!kubectl get hpa

In [ ]:
!kubectl get service nginx-service